In [ ]:
#!pip3 install -qU neptune

In [ ]:
import torch
import random
import numpy as np


SEED = 1000

def setup_reproducibility():
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(False, warn_only=True)
    torch.set_float32_matmul_precision("high")

setup_reproducibility()

In [ ]:
def return_stats(tensor, p=True, r=False):
    mean, std = tensor.mean(), tensor.std()
    min, max =  tensor.min(), tensor.max()
    
    if p: print(f"Min: {min}, Max: {max}, Mean: {mean}, Std: {std}")
    
    if r: return min, max, mean, std
    
    
def zscore(tensor, mean=None, std=None):
    if mean is None: mean = tensor.mean()
    if std is None: std = tensor.std()
    return (tensor - mean) / (std + 1e-6)


def get_model_size(model):
    print(sum(p.numel() for p in model.parameters()) / 1e6)

In [ ]:
import os

path = "/kaggle/input/dig-4-bio-raman-transfer-learning-challenge"
files = os.listdir(path)
[(i, files[i]) for i in range(len(files))]

In [ ]:
import pandas as pd


def load_transfer_data():
    csv_path = os.path.join(path, files[10])
    df = pd.read_csv(csv_path)

    input_cols = df.columns[1:2049]
    target_cols = df.columns[2050:]

    targets  = df[target_cols].dropna().to_numpy()

    df = df[input_cols]
    df['Unnamed: 1'] = df['Unnamed: 1'].str.replace("[\[\]]", "", regex=True).astype('int64')
    df['Unnamed: 2048'] = df['Unnamed: 2048'].str.replace("[\[\]]", "", regex=True).astype('int64')

    inputs = df.to_numpy().reshape(-1, 2, 2048)
    inputs = inputs.mean(axis=1)

    return inputs, targets





def load_all_datasets():
    train_inputs = []
    train_targets = []
    
    timegate = pd.read_csv(os.path.join(path, files[1]))

    timegate.drop(columns="fold_idx", inplace=True)
    timegate.drop(columns="MSM_present", inplace=True)
    timegate_inputs = timegate[timegate.columns[:-3]].to_numpy()
    timegate_targets = timegate[timegate.columns[-3:]].to_numpy()

    train_inputs.append(timegate_inputs)
    train_targets.append(timegate_targets)
    
    mettler_toledo = pd.read_csv(os.path.join(path, files[2]))

    mettler_toledo.drop(columns="fold_idx", inplace=True)
    mettler_toledo.drop(columns="MSM_present", inplace=True)
    mettler_toledo_inputs = mettler_toledo[mettler_toledo.columns[:-3]].to_numpy()
    mettler_toledo_targets = mettler_toledo[mettler_toledo.columns[-3:]].to_numpy()

    train_inputs.append(mettler_toledo_inputs)
    train_targets.append(mettler_toledo_targets)
    
    kaiser = pd.read_csv(os.path.join(path, files[3]))

    kaiser.drop(columns="fold_idx", inplace=True)
    kaiser.drop(columns="MSM_present", inplace=True)
    kaiser_inputs = kaiser[kaiser.columns[:-3]].to_numpy()
    kaiser_targets = kaiser[kaiser.columns[-3:]].to_numpy()

    train_inputs.append(kaiser_inputs)
    train_targets.append(kaiser_targets)
    
    anton = pd.read_csv(os.path.join(path, files[4]))

    anton.drop(columns="fold_idx", inplace=True)
    anton.drop(columns="MSM_present", inplace=True)
    anton_inputs = anton[anton.columns[:-3]].to_numpy()
    anton_targets = anton[anton.columns[-3:]].to_numpy()

    train_inputs.append(anton_inputs)
    train_targets.append(anton_targets)
    
    tornado = pd.read_csv(os.path.join(path, files[7]))

    tornado.drop(columns="fold_idx", inplace=True)
    tornado.drop(columns="MSM_present", inplace=True)
    tornado_inputs = tornado[tornado.columns[:-3]].to_numpy()
    tornado_targets = tornado[tornado.columns[-3:]].to_numpy()

    train_inputs.append(tornado_inputs)
    train_targets.append(tornado_targets)
    
    csv_path = os.path.join(path, files[8])
    print(csv_path)
    tec5 = pd.read_csv(csv_path)

    tec5.drop(columns="fold_idx", inplace=True)
    tec5.drop(columns="MSM_present", inplace=True)
    tec5_inputs = tec5[tec5.columns[:-3]].to_numpy()
    tec5_targets = tec5[tec5.columns[-3:]].to_numpy()

    train_inputs.append(tec5_inputs)
    train_targets.append(tec5_targets)
    
    csv_path = os.path.join(path, files[9])
    print(csv_path)
    metrohm = pd.read_csv(csv_path)

    metrohm.drop(columns="fold_idx", inplace=True)
    metrohm.drop(columns="MSM_present", inplace=True)
    metrohm_inputs = metrohm[metrohm.columns[:-3]].to_numpy()
    metrohm_targets = metrohm[metrohm.columns[-3:]].to_numpy()

    train_inputs.append(metrohm_inputs)
    train_targets.append(metrohm_targets)
    
    csv_path = os.path.join(path, files[10])
    print(csv_path)
    anton785 = pd.read_csv(csv_path)

    anton785.drop(columns="fold_idx", inplace=True)
    anton785.drop(columns="MSM_present", inplace=True)
    anton785_inputs = anton785[anton785.columns[:-3]].to_numpy()
    anton785_targets = anton785[anton785.columns[-3:]].to_numpy()

    train_inputs.append(anton785_inputs)
    train_targets.append(anton785_targets)
    
    return train_inputs, train_targets

train_inputs, train_targets = load_all_datasets()

In [ ]:
import torch
import torch.nn.functional as F


def foo(train_inputs, train_targets):
    for i in range(len(train_inputs)):
        x = train_inputs[i]
        x = torch.tensor(x).unsqueeze(0)
        x = F.interpolate(x, size=2048, mode="nearest-exact")
        train_inputs[i] = x.squeeze()
        
    train_inputs = torch.cat(train_inputs)
    train_targets = [torch.tensor(t) for t in train_targets]
    train_targets = torch.cat(train_targets)

    return train_inputs, train_targets
    
train_inputs, train_targets = foo(train_inputs, train_targets)
train_inputs.shape

In [ ]:
from scipy import signal
from tqdm.auto import tqdm


def get_advanced_spectra_features(X):
    """Create multi-channel features from spectra: raw, 1st derivative, 2nd derivative."""
    X_processed = np.zeros_like(X)
    # Baseline correction and SNV
    for i in tqdm(range(X.shape[0])):
        poly = np.polyfit(np.arange(X.shape[1]), X[i], 3)
        baseline = np.polyval(poly, np.arange(X.shape[1]))
        corrected_spec = X[i] - baseline
        X_processed[i] = (corrected_spec - corrected_spec.mean()) / (corrected_spec.std() + 1e-8)

    # Calculate derivatives
    deriv1 = signal.savgol_filter(X_processed, window_length=11, polyorder=3, deriv=1, axis=1)
    deriv2 = signal.savgol_filter(X_processed, window_length=11, polyorder=3, deriv=2, axis=1)

    # Stack as channels
    return np.stack([X_processed, deriv1, deriv2], axis=1)

#train_inputs = get_advanced_spectra_features(train_inputs.numpy())
#train_inputs = torch.tensor(train_inputs)

In [ ]:
#low = torch.quantile(train_inputs, 0.02, interpolation="linear")
#high = torch.quantile(train_inputs, 0.98, interpolation="linear")
#train_inputs = train_inputs.clamp(low, high)

In [ ]:
from sklearn.model_selection import train_test_split


train_inputs, eval_inputs, train_targets, eval_targets = train_test_split(
    train_inputs,
    train_targets,
    shuffle=True,
    test_size=0.2,
    random_state=1000
)

train_inputs.shape, eval_inputs.shape

In [ ]:
_, _, mean, std = return_stats(train_inputs, r=True)
train_inputs = zscore(train_inputs, mean, std)
return_stats(train_inputs), train_inputs.shape

In [ ]:
return_stats(eval_inputs)
eval_inputs = zscore(eval_inputs, mean, std)
return_stats(eval_inputs), eval_inputs.shape

In [ ]:
import pandas as pd


csv_path = os.path.join(path, files[5])
print(csv_path)
eval_set = pd.read_csv(csv_path)

eval_input_cols = eval_set.columns[1:2049]
eval_target_cols = eval_set.columns[2050:]

eval_targets  = eval_set[eval_target_cols].dropna().to_numpy()
eval_targets[:5]

eval_set = eval_set[eval_input_cols]
eval_set['Unnamed: 1'] = eval_set['Unnamed: 1'].str.replace('[\[\]]', '', regex=True).astype('int64')
eval_set['Unnamed: 2048'] = eval_set['Unnamed: 2048'].str.replace('[\[\]]', '', regex=True).astype('int64')

eval_inputs = eval_set.to_numpy().reshape(-1, 2, 2048)
eval_inputs = eval_inputs.mean(axis=1)
eval_inputs = get_advanced_spectra_features(eval_inputs)
eval_inputs.shape

In [ ]:
from sklearn.model_selection import train_test_split


train_inputs, eval_inputs, train_targets, eval_targets = train_test_split(
    eval_inputs,
    eval_targets,
    shuffle=True,
    test_size=0.2,
    random_state=1000
)

In [ ]:
train_inputs = torch.tensor(train_inputs).float()
train_targets = torch.tensor(train_targets).float()
eval_inputs = torch.tensor(eval_inputs).float()
eval_targets = torch.tensor(eval_targets).float()

In [ ]:
_, _, mean, std = return_stats(train_inputs, r=True)
train_inputs = zscore(train_inputs, mean, std)
return_stats(train_inputs)

In [ ]:
return_stats(eval_inputs)
eval_inputs = zscore(eval_inputs, mean, std)
return_stats(eval_inputs)

In [ ]:
train_inputs =  train_inputs.reshape(-1, 3 * 2048)

In [ ]:
eval_inputs = eval_inputs.reshape(-1, 3 * 2048)

In [ ]:
train_inputs = train_inputs.float().numpy()
train_targets = train_targets.float().numpy()
eval_inputs = eval_inputs.float().numpy()
eval_targets = eval_targets.float().numpy()

In [ ]:
import numpy as np
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score


xgb_base = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=150,      # fewer rounds because no early stopping; try 200-1000 depending on overfit
    learning_rate=0.06,
    max_depth=3,           # shallower trees for small data
    subsample=0.8,
    colsample_bytree=0.8,
    #reg_alpha=0.000001,        # L1 regularization
    reg_lambda=0.00001,        # L2 regularization
    random_state=SEED,
    verbosity=0            # quiet
)

# Wrap the XGB regressor so sklearn treats multi-targets automatically
multi_xgb = MultiOutputRegressor(xgb_base, n_jobs=-1)
multi_xgb.fit(train_inputs.astype(np.float32), train_targets.astype(np.float32))

preds = multi_xgb.predict(train_inputs)

scores = []
for i in range(3):
    scores.append(r2_score(train_targets[:, i], preds[:, i]))
print(scores)
print(np.stack(scores).mean())

preds = multi_xgb.predict(eval_inputs)

scores = []
for i in range(3):
    scores.append(r2_score(eval_targets[:, i], preds[:, i]))
print(scores)
np.stack(scores).mean()

In [ ]:
0.9017590299366272

In [ ]:
from sklearn.metrics import r2_score

preds = multi_xgb.predict(eval_inputs)

scores = []
for i in range(3):
    scores.append(r2_score(eval_targets[:, i], preds[:, i]))
print(scores)
np.stack(scores).mean()
from sklearn.metrics import r2_score

preds = multi_xgb.predict(eval_inputs)

scores = []
for i in range(3):
    scores.append(r2_score(eval_targets[:, i], preds[:, i]))
print(scores)
np.stack(scores).mean()

In [ ]:
from torch.utils.data import TensorDataset

train_ds = TensorDataset(train_inputs.float(), train_targets.float())
eval_ds = TensorDataset(eval_inputs.float(), eval_targets.float())

In [ ]:
from torch.utils.data import DataLoader


def build_loader(
    SEED,
    ds,
    train=True,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    drop_last=True,
    pin_memory=True,
    persistent_workers=False,
):
    def seed_worker(worker_id):
        worker_seed = torch.initial_seed() % 2**32
        np.random.seed(worker_seed)
        random.seed(worker_seed)

    generator = torch.Generator()
    generator.manual_seed(SEED if train else SEED+1)

    return DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        pin_memory=pin_memory,
        drop_last=drop_last,
        persistent_workers=persistent_workers,
        worker_init_fn=seed_worker,
        generator=generator,
        #sampler=DistributedSampler(
        #    train_ds,
        #    shuffle=True,
        #    drop_last=True,
        #    seed=config.seed
        #)
    )
    
    
def return_dls(train_ds, eval_ds):
    train_dl = build_loader(
        SEED,
        train_ds,
        train=True,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=0,
        drop_last=False,
        pin_memory=True,
        persistent_workers=False,
    )

    eval_dl = build_loader(
        SEED,
        eval_ds,
        train=False,
        batch_size=20,
        shuffle=False,
        num_workers=0,
        drop_last=False,
        pin_memory=True,
        persistent_workers=False,
    )
    
    return train_dl, eval_dl

In [ ]:
import neptune


def setup_neptune():
    if not RESUME:
        neptune_run = neptune.init_run(
            project="arbaaz/kaggle-spect",
            name=MODEL_NAME,
            api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJlOGE2YjNiZS1mZGUyLTRjYjItYTg5Yy1mZWJkZTIzNzE1NmIifQ=="
        )

        neptune_run["h_parameters"] = {
            "seed": SEED,
            "model_name": MODEL_NAME,
            "optimizer_name": "nadam",
            "learning_rate": LR,
            "scheduler_name": "default",
            "weight_decay": WD,
            "dropout": DROPOUT,
            "num_epochs": EPOCHS,
            "batch_size": BATCH_SIZE,
        }
        if DROP_PATH_RATE: neptune_run["h_parameters"] = {"drop_path_rate": DROP_PATH_RATE}
    else:
        neptune_run = neptune.init_run(
            project="arbaaz/crunchdao-structural-break",
            with_id=config.with_id,
            api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJlOGE2YjNiZS1mZGUyLTRjYjItYTg5Yy1mZWJkZTIzNzE1NmIifQ=="
        )

    return neptune_run

In [ ]:
import torch.nn.functional as F
from sklearn.metrics import r2_score


def loss_fn(logits, targets):
    logits = logits.view(-1)
    targets = targets.view(-1)
    return F.mse_loss(logits, targets)


def metric_fn(logits, targets):
    preds = logits.cpu().detach().numpy()
    targets = targets.cpu().detach().numpy()
    one = r2_score(targets[:, 0], preds[:, 0])
    two = r2_score(targets[:, 1], preds[:, 1])
    three = r2_score(targets[:, 2], preds[:, 2])
    mean_r2 = (one + two + three) / 3
    return one, two, three, mean_r2

In [ ]:
import torch.nn as nn


class ResidualBlock(nn.Module):
    """A residual block with two 1D convolutional layers."""
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding=kernel_size//2)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.elu = nn.ELU()
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, padding=kernel_size//2)
        self.bn2 = nn.BatchNorm1d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm1d(out_channels)
            )

    def forward(self, x):
        out = self.elu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.elu(out)
        return out

class ResNet(nn.Module):
    """A deeper ResNet-style 1D CNN for Raman spectra."""
    def __init__(self, input_channels=3, num_classes=3):
        super().__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm1d(64)
        self.elu = nn.GELU()
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(64, 2, stride=1)
        self.layer2 = self._make_layer(128, 2, stride=2)
        self.layer3 = self._make_layer(256, 2, stride=2)
        self.layer4 = self._make_layer(512, 2, stride=2)

        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.ELU(),
            nn.Dropout(0.5), # Increased dropout for better regularization
            nn.Linear(256, num_classes)
        )

    def _make_layer(self, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for s in strides:
            layers.append(ResidualBlock(self.in_channels, out_channels, stride=s))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.elu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [ ]:
import random
import warnings
from transformers import get_cosine_schedule_with_warmup
from tqdm.auto import tqdm

#warnings.filterwarnings("ignore")

MODEL_NAME = "ResNet.Finetune"
EPOCHS = 500
BATCH_SIZE = 19
WD = 1e-3
LR = 1e-4
DROPOUT = 0.5
DROP_PATH_RATE = 0.0
SCORE = float('-inf')
LOG = False
RESUME = False
device = "cuda" if torch.cuda.is_available() else "cpu"
checkpoint_name = "finetune.pt"


model = ResNet().to(device)
get_model_size(model)

ckpt_path = "/kaggle/working/pretrain.pt"
ckpt = torch.load(ckpt_path, weights_only=False)
model.load_state_dict(ckpt["state_dict"])

optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD, foreach=True)
scaler = torch.amp.GradScaler(device)
train_dl, eval_dl = return_dls(train_ds, eval_ds)

total_training_steps = len(train_dl) * EPOCHS
warmup_steps = int(total_training_steps * 0.05)  # e.g. 5% warmup
scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_training_steps
)

if LOG:
    neptune_run = setup_neptune()



for epoch in tqdm(range(EPOCHS)):
    model.train()
    total_loss = 0.0
    all_logits = []
    all_targets = []
    
    for inputs, targets in train_dl:
        inputs = inputs.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)
        
        #if random.random() < 0.7:
        #    noise = torch.randn_like(inputs) * 0.1  # std=0.1
        #    inputs = inputs + noise

        with torch.amp.autocast(device_type=device, dtype=torch.float16, cache_enabled=True):
            logits = model(inputs)
            loss = loss_fn(logits, targets)
        
        if LOG:  neptune_run["lr_step"].append(scheduler.get_last_lr()[0])
        scheduler.step()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        
        total_loss += loss.detach().cpu()
        all_logits.append(logits.detach().cpu())
        all_targets.append(targets.detach().cpu())
    
    all_logits = torch.cat(all_logits)
    all_targets = torch.cat(all_targets)

    one, two, three, r2 = metric_fn(all_logits, all_targets)
    total_loss = total_loss / len(train_dl)
    
    
    model.eval()
    eval_total_loss = 0.0
    eval_all_logits = []
    eval_all_targets = []

    for inputs, targets in eval_dl:
        inputs = inputs.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)

        with torch.inference_mode():
            with torch.amp.autocast(device_type=device, dtype=torch.float16, cache_enabled=True):
                logits = model(inputs)
                loss = loss_fn(logits, targets)

        eval_total_loss += loss.detach().cpu()
        eval_all_logits.append(logits.detach().cpu())
        eval_all_targets.append(targets.detach().cpu())
    
    eval_all_logits = torch.cat(eval_all_logits)
    eval_all_targets = torch.cat(eval_all_targets)

    eval_one, eval_two, eval_three, eval_r2 = metric_fn(eval_all_logits, eval_all_targets)
    eval_total_loss = eval_total_loss / len(eval_dl)
    
    if eval_r2 > SCORE:
        SCORE = eval_r2
        data = {"state_dict": model.state_dict()}
        data["epoch"] = epoch 
        data["score"] = SCORE
        torch.save(data, f"/kaggle/working/{checkpoint_name}")
    
    if LOG:
        neptune_run["train/loss"].append(total_loss)
        neptune_run["eval/loss"].append(eval_total_loss)
        neptune_run["train/r2"].append(r2)
        neptune_run["eval/r2"].append(eval_r2)
        neptune_run["train/one"].append(one)
        neptune_run["train/two"].append(two)
        neptune_run["train/three"].append(three)
        neptune_run["eval/one"].append(eval_one)
        neptune_run["eval/two"].append(eval_two)
        neptune_run["eval/three"].append(eval_three)
        
    if True:
        print(
            f"Epoch: {epoch}, "
            f"train/loss: {total_loss:.4f}, "
            f"eval/loss: {eval_total_loss:.4f}, "
            f"train/r2: {r2:.4f}, "
            f"eval/r2: {eval_r2:.4f}, "
            f"train/one: {one:.4f}, "
            f"train/two: {two:.4f}, "
            f"train/three: {three:.4f}, "
            f"eval/one: {eval_one:.4f}, "
            f"eval/two: {eval_two:.4f}, "
            f"eval/three: {eval_three:.4f} "
        )
        
if LOG:
    neptune_run.stop()

In [ ]:

import torch
ckpt_path = f"/kaggle/working/{checkpoint_name}"
ckpt = torch.load(ckpt_path, weights_only=False)
ckpt["epoch"], ckpt["score"]

In [ ]:

#def load_test():    
test = pd.read_csv(os.path.join(path, files[6]))

row1 = test.columns[1:].to_numpy().copy()
row1[-1] = "5611"
row1 = row1.astype(np.float64)


cols = test.columns[1:]
test = test[cols]
test[" 5611]"] = test[" 5611]"].str.replace('[\[\]]', '', regex=True).astype('int64')
test = test.to_numpy()

test = np.insert(test, 0, row1, axis=0)
test = test.reshape(-1, 2, 2048).mean(axis=1)

return_stats(test)
test = get_advanced_spectra_features(test)
test = torch.tensor(test)
return_stats(test)
test = zscore(test, mean, std).float()
test.shape, test.dtype, return_stats(test)

In [ ]:
test = test.numpy()
test = test.reshape(-1, 3 * 2048)
preds = multi_xgb.predict(test)
preds.shape

In [ ]:
preds = preds.astype(np.float64)

In [ ]:
preds.dtype

In [ ]:
device = "cuda"
model = ResNet().to(device)
model.load_state_dict(ckpt["state_dict"])
model.eval()

with torch.inference_mode():
    preds = model(test.cuda())

preds = preds.cpu().detach().numpy()
preds.shape

In [ ]:
#def prepare_test():
column_names = ['Glucose', 'Sodium Acetate', 'Magnesium Sulfate']
preds_df = pd.DataFrame(preds, columns=column_names)
preds_df.insert(0, 'ID', [i+1 for i in range(len(preds_df))])
preds_df

In [ ]:
MODEL_NAME

In [ ]:
#def save_test():
#name = f"{MODEL_NAME}.{ckpt['score']:.4f}.csv"
name = "xgboost.concat.9017.csv"
preds_df.to_csv(name, index=False)
f = pd.read_csv(f"/kaggle/working/{name}")
f

In [ ]:
name